In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
df_main = pd.read_hdf('../../data/processed/solicitacoes156.h5', key='data', format="table")
df_socio = pd.read_hdf('../../data/processed/socioeconomico.h5', key='data', format="table")

In [3]:
df_socio.head()

,BAIRRO,RPA,MICROREGIAO,DIST_MARCO_ZERO,HECTARES,HABITANTES,NUM_HOMENS,POR_HOMENS,NUM_MULHERES,POR_MULHERES,...,PARDA,AMARELA,INDIGENA,ALFABETIZADOS,CRESCIMENTO_ANUAL,DENSIDADE,DOMICILIOS,MEDIA_MORADORES,MULHERES_RESPONSAVEIS,RENDA
0,RECIFE,1,1.1,0.00,270.0,602,287,47.67,315,52.33,...,30.72,0.50,0.50,81.7,-4.20,2.23,198,3.0,66.83,567.00
1,BOA VISTA,1,1.2,2.78,176.0,14778,6164,41.71,8614,58.29,...,33.37,1.25,0.26,98.0,0.52,84.15,5999,2.4,54.43,3618.45
2,CABANGA,1,1.2,3.26,81.0,1551,679,43.78,872,56.22,...,45.13,1.35,0.32,94.0,0.10,19.16,506,3.1,57.00,1986.08
3,COELHOS,1,1.3,2.15,43.0,7633,3571,46.78,4062,53.22,...,53.36,0.86,0.72,87.0,1.12,178.51,2322,3.6,53.63,898.41
4,ILHA DO LEITE,1,1.2,2.64,26.0,1007,446,4.29,561,55.71,...,33.86,1.49,0.40,96.0,0.49,38.21,361,2.8,56.79,3024.11


In [4]:
df_socio.columns

Index(['BAIRRO', 'RPA', 'MICROREGIAO', 'DIST_MARCO_ZERO', 'HECTARES',
       'HABITANTES', 'NUM_HOMENS', 'POR_HOMENS', 'NUM_MULHERES',
       'POR_MULHERES', '0_4ANOS', '0_4ANOS%', '5_14ANOS', '5_14ANOS%',
       '15_17ANOS', '15_17ANOS%', '18_24ANOS', '18_24ANOS%', '25_59ANOS',
       '25_59ANOS%', '60_ANOS', '60_ANOS%', 'BRANCA', 'PRETA', 'PARDA',
       'AMARELA', 'INDIGENA', 'ALFABETIZADOS', 'CRESCIMENTO_ANUAL',
       'DENSIDADE', 'DOMICILIOS', 'MEDIA_MORADORES', 'MULHERES_RESPONSAVEIS',
       'RENDA'],
      dtype='object')

#### O dataset de perfil socioeconomico é a primeira informação externa a ser utilizada no enriquecimentos dos dados. E possui as colunas vistas acima. Desses dados, o único que não possui representatividade, ou seja, que não agrega informação útil é o dado de RPA. Aqui RPA representa apenas a zona politico-admistrativa do bairro.

In [5]:
df_socio.drop(["RPA","MICROREGIAO","DIST_MARCO_ZERO"],axis = 1, inplace = True)

##### A união dos datasets será atraves da coluna de BAIRRO

In [6]:
df_main = pd.merge(df_main,df_socio,on="BAIRRO")

##### Para saber mais sobre os dados e o perfil socioeconomico vamos fazer uma análise vais detalhada desses dados.

* Primeiramente, vamos análisar alguns dados simples, como a amplitude de renda, ou seja, a diferença entre os que ganham mais e ganham menos, bem como o desvio padrão desses valores

In [7]:
amplitude = df_socio.RENDA.max() - df_socio.RENDA.min()
print(amplitude)

10772.79


##### Esse valor, apesar de ser obtido de forma simples, revela uma desigualdade absurda. A diferença  entre a renda do bairro mais rico e do mais pobre é de 10772,79 reais. Quando temos em mente que o calculo de renda utilizado pelo senso e disponibilizado pela prefeitura, é o da renda média, essa diferença se torna ainda mais gritante.

In [8]:
desvio = df_socio.RENDA.std()
print(desvio)

2659.8160252327307


##### Valores altos para o desvio padrão significa que os dados estão bastant dispersos (desigualdade), uma vez que essa medida representao quanto os dados estão afastados da media.

* Apesar de não ser o foco dessa análise, vale a pena verificar a correlação entre elementos já conhecidos por serem decisivos na renda e na qualidade de vida das pessoas.

In [9]:
df_socio[["ALFABETIZADOS", "RENDA"]].corr()

,ALFABETIZADOS,RENDA
ALFABETIZADOS,1.000000,0.665981
RENDA,0.665981,1.000000


In [10]:
df_socio[['CRESCIMENTO_ANUAL','DENSIDADE', 'DOMICILIOS', 'MEDIA_MORADORES','ALFABETIZADOS','RENDA']].corr()

,CRESCIMENTO_ANUAL,DENSIDADE,DOMICILIOS,MEDIA_MORADORES,ALFABETIZADOS,RENDA
CRESCIMENTO_ANUAL,1.000000,0.058958,0.011842,-0.140382,0.027258,0.106767
DENSIDADE,0.058958,1.000000,0.028418,0.113014,-0.177959,-0.115106
DOMICILIOS,0.011842,0.028418,1.000000,-0.011380,-0.000396,-0.051503
MEDIA_MORADORES,-0.140382,0.113014,-0.011380,1.000000,-0.498607,-0.279802
ALFABETIZADOS,0.027258,-0.177959,-0.000396,-0.498607,1.000000,0.665981
RENDA,0.106767,-0.115106,-0.051503,-0.279802,0.665981,1.000000


In [11]:
df_socio[['RENDA', 'POR_HOMENS', 'POR_MULHERES']].corr()

,RENDA,POR_HOMENS,POR_MULHERES
RENDA,1.000000,-0.218535,0.412997
POR_HOMENS,-0.218535,1.000000,-0.582470
POR_MULHERES,0.412997,-0.582470,1.000000


In [12]:
df_socio[['0_4ANOS%','5_14ANOS%','15_17ANOS%', '18_24ANOS%',
       '25_59ANOS%', '60_ANOS%','ALFABETIZADOS','RENDA']].corr()

,0_4ANOS%,5_14ANOS%,15_17ANOS%,18_24ANOS%,25_59ANOS%,60_ANOS%,ALFABETIZADOS,RENDA
0_4ANOS%,1.000000,0.905832,0.725579,0.041118,-0.333134,-0.848477,-0.825135,-0.504302
5_14ANOS%,0.905832,1.000000,0.871882,0.078632,-0.367736,-0.902998,-0.889233,-0.612101
15_17ANOS%,0.725579,0.871882,1.000000,0.199818,-0.417778,-0.836319,-0.777497,-0.630153
18_24ANOS%,0.041118,0.078632,0.199818,1.000000,-0.110465,-0.283851,-0.219090,-0.263413
25_59ANOS%,-0.333134,-0.367736,-0.417778,-0.110465,1.000000,0.304931,0.384137,0.298585
60_ANOS%,-0.848477,-0.902998,-0.836319,-0.283851,0.304931,1.000000,0.849175,0.649331
ALFABETIZADOS,-0.825135,-0.889233,-0.777497,-0.219090,0.384137,0.849175,1.000000,0.665981
RENDA,-0.504302,-0.612101,-0.630153,-0.263413,0.298585,0.649331,0.665981,1.000000


#### Apesar da correlação indicar uma relação entre 2 valores, isso não quer dizer necessáriamente que essa relação existe de fato. Para o exemplos mostrados acima pode-se fazer alguns comentarios.
 * Uma relação forte, mais proxima de 1, entre renda e a porcentagem de alfabetizados é uma relação que faz sentido e tende a ser verdadeira, visto que níveis mais elevados e educação propiciam melhores ganhos, consequentimente aumento de renda.
 * Existe uma relação mais fraca entre renda e crecimento anual da população. Essa relação pode ser mais fraca devido ao fato de que o crescimento populacional não impacta a curto prazo a renda media do local.
 * Ao analisar a renda com a porcentagem de homem e mulheres, vemos um impacto das mulheres na renda um tanto significativo.
 * Já para o porcentagem por faixa de idade, temos um fato curioso. A renda e a taxa de alfabetizados, tem relação apenas para a faixa entre 25 e 60 anos. O que indica que melhores condições de vida, como bons ganhos e uma educação básica, podem influenciar na longevidade de uma população.

### Agora que já foi feita uma breve análise sobre os dados socioeconomicos, vamos seguir com o desenvolvimento da hipotese, e verificar se existe relação do perfil socioeconomico do bairro com o tipo de solicitação.

In [13]:
df_main.columns

Index(['GRUPOSERVICO_CODIGO', 'GRUPOSERVICO_DESCRICAO', 'SERVICO_CODIGO',
       'SERVICO_DESCRICAO', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'RPA',
       'DATA_DEMANDA', 'SITUACAO', 'DATA_ULT_SITUACAO', 'ANO', 'HECTARES',
       'HABITANTES', 'NUM_HOMENS', 'POR_HOMENS', 'NUM_MULHERES',
       'POR_MULHERES', '0_4ANOS', '0_4ANOS%', '5_14ANOS', '5_14ANOS%',
       '15_17ANOS', '15_17ANOS%', '18_24ANOS', '18_24ANOS%', '25_59ANOS',
       '25_59ANOS%', '60_ANOS', '60_ANOS%', 'BRANCA', 'PRETA', 'PARDA',
       'AMARELA', 'INDIGENA', 'ALFABETIZADOS', 'CRESCIMENTO_ANUAL',
       'DENSIDADE', 'DOMICILIOS', 'MEDIA_MORADORES', 'MULHERES_RESPONSAVEIS',
       'RENDA'],
      dtype='object')

In [27]:
df_main[['SERVICO_CODIGO','GRUPOSERVICO_CODIGO']] = df_main[['SERVICO_CODIGO','GRUPOSERVICO_CODIGO']].astype('int64')

In [28]:
df_main[['SERVICO_CODIGO','RENDA']].corr()

,SERVICO_CODIGO,RENDA
SERVICO_CODIGO,1.000000,0.007016
RENDA,0.007016,1.000000


In [31]:
df_main[['GRUPOSERVICO_CODIGO','RENDA']].corr()

,GRUPOSERVICO_CODIGO,RENDA
GRUPOSERVICO_CODIGO,1.000000,-0.059987
RENDA,-0.059987,1.000000


In [35]:
df_main[['SERVICO_CODIGO','ALFABETIZADOS']].corr()

,SERVICO_CODIGO,ALFABETIZADOS
SERVICO_CODIGO,1.000000,0.002029
ALFABETIZADOS,0.002029,1.000000


In [37]:
df_main[['0_4ANOS%','5_14ANOS%','15_17ANOS%', '18_24ANOS%',
       '25_59ANOS%', '60_ANOS%','ALFABETIZADOS','SERVICO_CODIGO']].corr()

,0_4ANOS%,5_14ANOS%,15_17ANOS%,18_24ANOS%,25_59ANOS%,60_ANOS%,ALFABETIZADOS,SERVICO_CODIGO
0_4ANOS%,1.000000,0.908690,0.802983,0.369980,-0.439294,-0.876118,-0.853660,0.000262
5_14ANOS%,0.908690,1.000000,0.919645,0.400150,-0.492061,-0.918466,-0.894035,0.001651
15_17ANOS%,0.802983,0.919645,1.000000,0.487093,-0.527594,-0.893134,-0.830501,-0.002573
18_24ANOS%,0.369980,0.400150,0.487093,1.000000,-0.156406,-0.519235,-0.387312,-0.001496
25_59ANOS%,-0.439294,-0.492061,-0.527594,-0.156406,1.000000,0.414674,0.494120,0.001542
60_ANOS%,-0.876118,-0.918466,-0.893134,-0.519235,0.414674,1.000000,0.839103,0.000640
ALFABETIZADOS,-0.853660,-0.894035,-0.830501,-0.387312,0.494120,0.839103,1.000000,0.002029
SERVICO_CODIGO,0.000262,0.001651,-0.002573,-0.001496,0.001542,0.000640,0.002029,1.000000


In [39]:
df_main[['CRESCIMENTO_ANUAL','DENSIDADE', 'DOMICILIOS', 'MEDIA_MORADORES','ALFABETIZADOS','SERVICO_CODIGO']].corr()

,CRESCIMENTO_ANUAL,DENSIDADE,DOMICILIOS,MEDIA_MORADORES,ALFABETIZADOS,SERVICO_CODIGO
CRESCIMENTO_ANUAL,1.000000,0.044680,0.116335,-0.088988,0.045214,0.001144
DENSIDADE,0.044680,1.000000,-0.030867,0.131625,-0.148691,-0.001125
DOMICILIOS,0.116335,-0.030867,1.000000,-0.159692,0.249853,-0.017694
MEDIA_MORADORES,-0.088988,0.131625,-0.159692,1.000000,-0.544455,0.008702
ALFABETIZADOS,0.045214,-0.148691,0.249853,-0.544455,1.000000,0.002029
SERVICO_CODIGO,0.001144,-0.001125,-0.017694,0.008702,0.002029,1.000000


In [40]:
df_main[['SERVICO_CODIGO', 'POR_HOMENS', 'POR_MULHERES']].corr()

,SERVICO_CODIGO,POR_HOMENS,POR_MULHERES
SERVICO_CODIGO,1.000000,-0.001220,0.004292
POR_HOMENS,-0.001220,1.000000,-0.646508
POR_MULHERES,0.004292,-0.646508,1.000000


##### Com a correlações encontradas, não é possível verificar um relação forte entre a renda do bairro , e questões sociais com o tipo de solicitação feita. Esse fato pode ser observado no gráfico geografico encontrado no Tableau. Podemos notar, que os tipos de solicitação estão bem dispostos entre os diferentes bairros do Recife. Não havendo uma aglomeração especifica que permita identificar